In [1]:
import tensorflow as tf
import numpy
from matplotlib import pyplot as plt
from keras.datasets import fashion_mnist as fmnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D


<h3>Loading the train test splits for fashion_mnist</h3>

In [2]:
(train_X, train_y), (test_X, test_y) = fmnist.load_data()

<h3>Preprocessing data</h3>

In [3]:
def normalize(image):
    
    image = tf.cast(image, dtype=float)/255
    image = tf.reshape(image, [-1,])

    return image

train_X = tf.data.Dataset.from_tensor_slices(train_X)
train_y = tf.data.Dataset.from_tensor_slices(train_y)

test_X = tf.data.Dataset.from_tensor_slices(test_X)
test_y = tf.data.Dataset.from_tensor_slices(test_y)

new_train_X = train_X.map(normalize)
new_test_X = test_X.map(normalize)

def one_hot_matrix(labels, depth = 10):
    
    one_hot = tf.reshape(tf.one_hot(labels, depth, axis = 0), [-1,])
    
    return one_hot

new_train_y = train_y.map(one_hot_matrix)
new_test_y = test_y.map(one_hot_matrix)

<h3>Creating a Model</h3>